# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [76]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.tile_sources as gvts
import numpy as np
import geopandas as gpd

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,284.27,71,91,7.15,NZ,1709921136
1,1,yeniseysk,58.4497,92.1703,250.34,100,87,0.89,RU,1709921137
2,2,badger,64.8000,-147.5333,255.03,63,75,0.00,US,1709921139
3,3,wailua homesteads,22.0669,-159.3780,294.12,77,75,9.77,US,1709921140
4,4,trujillo,-8.1160,-79.0300,298.13,78,0,5.66,PE,1709921141


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

city_data_df['Humidity'] = pd.to_numeric(city_data_df['Humidity'], errors='coerce')

# Create a GeoViews Points object with size based on 'Humidity'
points = gv.Points(city_data_df, ['Lng', 'Lat'], ['City', 'Humidity']).options(size='Humidity')

# Create the map plot with points sized by humidity
map_plot = gvts.CartoLight * points.opts(tools=['hover'])

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = {
    'Temperature': {'max': 300},  # Roughly 80F
    'Humidity': {'max': 30},  # Maximum humidity percentage
    'Cloudiness': {'max': 30}  # Maximum cloudiness percentage
}

# Drop any rows with null values
city_data_df_new = city_data_df.dropna()

# Filter the DataFrame based on the ideal conditions
ideal_cities_df = city_data_df_new[
    (city_data_df_new['Max Temp'] <= ideal_conditions['Temperature']['max']) & 
    (city_data_df_new['Humidity'] <= ideal_conditions['Humidity']['max']) & 
    (city_data_df_new['Cloudiness'] <= ideal_conditions['Cloudiness']['max'])
]

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,tura,25.5198,90.2201,290.68,28,0,1.14,IN,1709921186
81,81,mhamid,29.8200,-5.7200,296.58,15,0,4.22,MA,1709921245
88,88,xinzhou,38.4092,112.7333,274.08,30,0,0.32,CN,1709921264
94,94,at taj,24.2049,23.2857,297.85,13,0,5.75,LY,1709921283
167,167,serilingampalle,17.4931,78.3020,299.49,22,1,2.08,IN,1709921115
370,370,orkney,-26.9802,26.6727,297.92,14,1,3.21,ZA,1709921721
472,472,anupgarh,29.1911,73.2086,291.29,22,1,3.39,IN,1709921883
475,475,kebili,33.7044,8.9690,295.13,22,0,2.57,TN,1709921887
484,484,ha'il,27.5219,41.6907,291.16,29,0,3.09,SA,1709921900
522,522,ghat,24.9647,10.1728,298.58,11,0,3.10,LY,1709921960


### Step 3: Create a new DataFrame called `hotel_df`.

In [83]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.loc[:, ['City', 'Country', 'Lat', 'Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
38,tura,IN,25.5198,90.2201,28,
81,mhamid,MA,29.8200,-5.7200,15,
88,xinzhou,CN,38.4092,112.7333,30,
94,at taj,LY,24.2049,23.2857,13,
167,serilingampalle,IN,17.4931,78.3020,22,
370,orkney,ZA,-26.9802,26.6727,14,
472,anupgarh,IN,29.1911,73.2086,22,
475,kebili,TN,33.7044,8.9690,22,
484,ha'il,SA,27.5219,41.6907,29,
522,ghat,LY,24.9647,10.1728,11,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [74]:
# Set parameters to search for a hotel

radius = 10000
categories = "accommodation.hotel"
limit = 1

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: Hotel Polo Orchid
mhamid - nearest hotel: Hotel Kasbah Azalay
xinzhou - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
serilingampalle - nearest hotel: Brundavanam Hotel
orkney - nearest hotel: No hotel found
anupgarh - nearest hotel: Hotel Namish
kebili - nearest hotel: Hôtel du Autruches
ha'il - nearest hotel: فندق كلاسيك
ghat - nearest hotel: فندق تاسيلي
mathba - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة


,City,Country,Lat,Lng,Humidity,Hotel Name
38,tura,IN,25.5198,90.2201,28,Hotel Polo Orchid
81,mhamid,MA,29.8200,-5.7200,15,Hotel Kasbah Azalay
88,xinzhou,CN,38.4092,112.7333,30,No hotel found
94,at taj,LY,24.2049,23.2857,13,No hotel found
167,serilingampalle,IN,17.4931,78.3020,22,Brundavanam Hotel
370,orkney,ZA,-26.9802,26.6727,14,No hotel found
472,anupgarh,IN,29.1911,73.2086,22,Hotel Namish
475,kebili,TN,33.7044,8.9690,22,Hôtel du Autruches
484,ha'il,SA,27.5219,41.6907,29,فندق كلاسيك
522,ghat,LY,24.9647,10.1728,11,فندق تاسيلي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [79]:
%%capture --no-display

# Create a GeoDataFrame from the data
gdf = gpd.GeoDataFrame(hotel_df, geometry=gpd.points_from_xy(hotel_df['Lng'], hotel_df['Lat']))

# Convert GeoDataFrame to a pandas DataFrame
df = pd.DataFrame(gdf.drop(columns='geometry'))

# Create a Points plot with customized tooltips
points = gv.Points(df, ['Lng', 'Lat'], ['City', 'Humidity', 'Hotel Name', 'Country']).opts(
    tools=['hover'], size=10, color='blue', cmap='viridis', width=600, height=400,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map with the Points plot
tiles = gvts.CartoLight.options(width=600, height=400)
tiles * points

ValueError: Unexpected option 'hover_cols' for Points type across all extensions. Similar options for current extension ('bokeh') are: ['hover_color', 'hover_fill_color', 'hover_line_color'].